# Seminar 1: Solutions to Exercises

**LSE MY459: Computational Text Analysis and Large Language Models** (WT 2026)

**Ryan Hübert**

## Part 1. Directory Management

1. In the following code chunk, create an object called `edir` that has the fill path to the directory where you will store your work on these exercises.

In [1]:
import os
edir = os.path.join(os.path.expanduser("~"), "LSE-MY459-WT26", "SeminarWeek02")

2. In the following code chunk, download the file `news_article.txt` from the data repo on the course GitHub, and save it into `edir`.

In [2]:
import requests

rfile = "https://raw.githubusercontent.com/lse-my459/data/master/news_article.txt"

# Where will we store the local copy of it?
lfile = os.path.join(edir, os.path.basename(rfile))

# Check if you have the file yet and if not, download it to correct location
if not os.path.exists(lfile):
    r = requests.get(rfile) # make GET request for the remote file
    r.raise_for_status()    # raise exception if there's an HTTP error
    
    # Write the raw bytes received from the server to the local file path
    with open(lfile, "wb") as f:
        f.write(r.content)

## Part 2: Fun with Digital Text

1. Read `news_article.txt` into Python as raw bytes, and then echo the first 10 bytes of the resulting `bytes` object.

In [3]:
with open(lfile, 'rb') as f:
    raw = f.read()
raw[0:10]

b'"\xe6\xc1\xce\xd4\xc1\xda\xc9\xc9 '

2. Try to decode the `bytes` object using the UTF-8 and UTF-16 encoding standards and the `bytes.decode()` method. Is the file you read into Python UTF-8 or UTF-16 encoded?

In [9]:
# Note: this try-except will try to decode, and if there is an error, it prints the error. You can also just have `raw.decode("utf-8")` in this chunk and see it returns 
# an error, although it would create some complications for running all cells in this file at once (due to the error)
try:
    raw.decode("utf-8")
except UnicodeDecodeError as e:
    print(f"This data is not UTF-8 encoded, see following error: {e}")

This data is not UTF-8 encoded, see following error: 'utf-8' codec can't decode byte 0xe6 in position 1: invalid continuation byte


In [ ]:
try:
    raw.decode("utf-16")
except UnicodeDecodeError as e:
    print(f"This data is not UTF-16 encoded, see following error: {e}")

This data is not UTF-16 encoded, see following error: 'utf-16-le' codec can't decode bytes in position 26-27: illegal UTF-16 surrogate


3. Now, use the `charset_normalizer` module to try to discover the encoding.

In [11]:
import charset_normalizer
print(charset_normalizer.detect(raw))

{'encoding': 'KOI8-R', 'language': 'Bulgarian', 'confidence': 0.995}


4. Now try to decode the `bytes` object using the best guess you get from the `charset_normalizer` module. Print the decoded text using the `print()` function.

In [13]:
print(raw.decode(encoding='KOI8-R'))

"Фантазии единственная реальность": 105 лет со дня рождения Федерико Феллини
105 лет назад родился Федерико Феллини
20 января 2025, 09:30
20 января исполняется 105 лет со дня рождения культового итальянского режиссёра Федерико Феллини. Большое влияние на его становление оказали цирковое искусство и графика: долгое время Феллини трудился карикатуристом. Его карьера в кино началась с написания сценариев, в том числе к одному из фильмов Роберто Росселлини. Первые режиссёрские работы Феллини были встречены овациями на фестивалях, а ленты "Ночи Кабирии", "Репетиция оркестра", "8 1/2" и "Амаркорд" стали классикой мирового кино и ориентиром для последующих поколений кинематографистов. Федерико Феллини обладатель пяти премий "Оскар" и лауреат Каннского и Венецианского кинофестивалей. По сей день он считается одним из величайших режиссёров в истории.


5. Save a new version of the file called `news_article_utf8.txt`, where the text you've just decoded is re-encoded as UTF-8 when saving.

In [14]:
with open(lfile.replace(".txt", "utf8.txt"), 'w', encoding='utf-8') as f:
    f.write(raw.decode(encoding='KOI8-R'))

6. Try to read the newly created file in to Python to verify that it is indeed encoded as UTF-8.

In [15]:
## If no error, then it is correct!
utf8_file = lfile.replace(".txt", "utf8.txt")
with open(utf8_file, 'r', encoding="utf-8") as f:
    print(f.read()[0:100])

"Фантазии единственная реальность": 105 лет со дня рождения Федерико Феллини
105 лет назад родился Ф


7. How many characters is in this file? How many bytes does it take to store it as a UTF-8 encoded file?

In [16]:
## Count characters by counting length of string after decoding
print(len(open(utf8_file, 'r').read())) # read and decode bytes and calculate len()

## Get size in two ways
## Option 1:
print(os.path.getsize(utf8_file))
## Option 2:
print(len(open(utf8_file, 'rb').read())) # read raw bytes and calculate len()

853
1539
1539


## Part 3: Loading Trump Tweet Data

1. Load the sparse matrix, vocabular and the tweet corpus you made in a previous notebook.

In [17]:
from scipy import sparse
import pandas as pd

dfm_sparse = sparse.load_npz(os.path.join(edir, 'tweet-dfm-sparse.npz'))
vocabulary = open(os.path.join(edir, 'tweet-dfm-features.txt'), mode = "r").read().split("\n")
corpus = pd.read_csv(os.path.join(edir, "tweet-corpus.csv"))
corpus["datetime"].head()
corpus["datetime"] = pd.to_datetime(corpus["datetime"]).dt.tz_convert("US/Eastern") # Convert to US Eastern time

2. Remove any tweet posted before Trump became president. Hint: all US presidents are inaugurated at 12:00 Eastern US time on the 20th of January of their first year in office.

In [18]:
corpus = corpus.loc[corpus["datetime"] >= pd.to_datetime("2017-01-20 12:00-05:00"),:]
print(corpus.shape)

(3722, 2)


In [19]:
idx = corpus.index.to_numpy()
dfm_sparse = dfm_sparse[idx,:]

3. Trim the dataset features by dropping any token that does not occur in at least 5 documents, and at least 8 times total in the corpus. How many features did you trim out?

In [20]:
import numpy as np 

docf = (dfm_sparse > 0).sum(axis=0).A1
ttf = dfm_sparse.sum(axis=0).A1

min_docf = 5
min_ttf = 8

cols_keep = np.where((docf >= min_docf) & (ttf >= min_ttf))[0]
print(f"Removed {dfm_sparse.shape[1] - len(cols_keep)} features.")

vocabulary_trimmed = [vocabulary[x] for x in cols_keep]
dfm_trimmed = dfm_sparse[:, cols_keep]
print(f"New DFM has shape: {dfm_trimmed.shape}")

Removed 4770 features.
New DFM has shape: (3722, 1179)


## Part 4: Basic Text Manipulations

1. Display Trump's first tweet as president. Do this in three chunks: one where you echo the tweet, one where you print the tweet, and one where you pretty print the tweet.

In [21]:
corpus["text"].iloc[0]

'Today we are not merely transferring power from one Administration to another, or from one party to another – but we are transferring...'

In [22]:
print(corpus["text"].iloc[0])

Today we are not merely transferring power from one Administration to another, or from one party to another – but we are transferring...


In [23]:
from pprint import pprint
pprint(corpus["text"].iloc[0])

('Today we are not merely transferring power from one Administration to '
 'another, or from one party to another – but we are transferring...')


2. Find the tweet posted at 12:55 pm Eastern US time on the day that Trump was first inaugurated.

In [24]:
# Find tweet at 17:55 UK time on Jan 20, 2017
tweet = corpus.loc[corpus["datetime"] >= pd.to_datetime("2017-01-20 12:55-05:00"),:]
tweet = tweet["text"].iloc[0]
print(tweet)

We will follow two simple rules: BUY AMERICAN &amp; HIRE AMERICAN!
#InaugurationDay #MAGA🇺🇸


3. Tokenise the tweet using whitespace.

In [25]:
import re
tok = re.split(r'\s+', tweet)
print(tok)

['We', 'will', 'follow', 'two', 'simple', 'rules:', 'BUY', 'AMERICAN', '&amp;', 'HIRE', 'AMERICAN!', '#InaugurationDay', '#MAGA🇺🇸']


4. Replace any emojis with a descriptive placeholder.

In [26]:
# Replace emoji patterns (MAGA flag emoji example)
tok = [re.sub(r'[#]MAGA.+', '#MAGA<us_flag_emoji>', t) for t in tok]
print(tok)

['We', 'will', 'follow', 'two', 'simple', 'rules:', 'BUY', 'AMERICAN', '&amp;', 'HIRE', 'AMERICAN!', '#InaugurationDay', '#MAGA<us_flag_emoji>']


5. Pre-process the tweet by: standardising the cases as appropriate, cleaning out non-words, punctuation and numbers, and any other step you need to take.

In [27]:
# Make lowercase
tok = [x.lower() for x in tok]
    
# Remove HTML entities like &amp; &lt; etc.
tok = [x for x in tok if not re.match(r'^&#?[a-z]+;$',x)]

# Remove punctuation but keep # < >
tok = [re.sub(r'[^a-z#<> ]', '', x) for x in tok]
    
# Remove excess whitespace
tok = [x.strip() for x in tok]
    
# Remove empty strings
tok = [t for t in tok if t]  

print(tok)

['we', 'will', 'follow', 'two', 'simple', 'rules', 'buy', 'american', 'hire', 'american', '#inaugurationday', '#maga<usflagemoji>']


6. Load English stop words from the `nltk` module and remove any stop word from the tokens.

In [28]:
from nltk.corpus import stopwords
sw = stopwords.words('english')
tok = [x for x in tok if x not in sw]
print(tok)

['follow', 'two', 'simple', 'rules', 'buy', 'american', 'hire', 'american', '#inaugurationday', '#maga<usflagemoji>']


7. Stem words using the Snowball stemmer.

In [29]:
from nltk.stem import snowball
sstemmer = snowball.SnowballStemmer("english")
tok = [sstemmer.stem(x) for x in tok]
print(tok)

['follow', 'two', 'simpl', 'rule', 'buy', 'american', 'hire', 'american', '#inaugurationday', '#maga<usflagemoji>']


8. Using a `Counter` object, calculate the term frequencies of this tweet. Echo the `Counter` object.

In [30]:
from collections import Counter
tok_count = Counter(tok)
tok_count

Counter({'american': 2,
         'follow': 1,
         'two': 1,
         'simpl': 1,
         'rule': 1,
         'buy': 1,
         'hire': 1,
         '#inaugurationday': 1,
         '#maga<usflagemoji>': 1})

9. Turn this `Counter` object into a row of a Pandas `DataFrame` object and into a row of a SciPy sparse array object.

In [31]:
pd.DataFrame([tok_count])

,follow,two,simpl,rule,buy,american,hire,#inaugurationday,#maga<usflagemoji>
0,1,1,1,1,1,2,1,1,1


In [33]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()
print(dv.fit_transform(tok_count))

vocabulary = dv.get_feature_names_out() # preserve the vocab!
vocabulary

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (1, 9)>
  Coords	Values
  (0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	2.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0


array(['#inaugurationday', '#maga<usflagemoji>', 'american', 'buy',
       'follow', 'hire', 'rule', 'simpl', 'two'], dtype=object)